<h2><center>Exercise 02</center>
    
<center>Potential energy minimisation</center>

<center>short</center></h2>

2.1 Energy surface

Consider a function given by

\begin{equation}
	U\left(x,y\right) = \left( x- y \right)^4 + 2 x^2 + y^2 - x + 2y
\end{equation}

2.1.1 Calculate the gradient and the hessian matrix of $U\left(x,y\right)$.

2.1.2 Write a Python-script that finds a local minimum of the function by the steepest
decent method. Please use section 2.2.3 and equation 2.8 (page 45/46 in the script)
as orientation. The algorithm should stop, when the energy correction (eps) is
smaller than $10^{-10}$ or the number of iteration steps n reaches 1000. Show a table for
the starting points (1, 1), (0, 0) and (−0.3, 3), which holds the number of iterations
n and the corresponding eps. Choose $\tau = 0.09$.

General minimisation approach:
$$ q_{k+1} = q_k + \tau_k d_k $$

#### Steepest decent

Direction for minimisation along the gradient:
$$ d_k = -g_k $$
$$ q_{k+1} = q_k - \tau g_k $$

2.1.3 Create another table for the starting point (1, 1) and choose three different values
for $\tau$.

2.1.4 Repeat task 2., but this time for the conjugate gradient method (page 47 in the
script).

#### Conjugate gradient

Direction for minimisation is determined by considering the direction and gradient used in the last step (first step needs to use a different algorithm).

$$ d_k = −g_k + \beta_k d_{k−1} $$

Expression for $\beta$ (Fletcher-Reeves):
$$ \beta_k = \frac{g_k^\mathrm{T} g_k}{g_{k-1}^\mathrm{T} g_{k-1}^\mathrm{\phantom{T}}} $$

2.1.5 Draw for each of the starting points the sequence of iteration points into a contour
diagram of the minimised function.


In [40]:
%matplotlib notebook
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import sys


In [3]:
def U(q):
    # 2D-potential
    return (q[0] - q[1])**4 + 2*q[0]**2 + q[1]**2 - q[0] + 2*q[1]

In [6]:
def gradient(q):
    # gradient of U
    return (4 * (q[0] - q[1])**3 + 4*q[0] - 1,
            - 4 * (q[0] - q[1])**3 + 2*q[1] + 2)

In [7]:
def hessian(q):
    #hessian of U
    return (( 12*(q[0] - q[1])**2 + 4,  -12*(q[0] - q[1])**2 ),
             ( -12*(q[0] - q[1])**2, 12*(q[0] - q[1])**2 + 2))

In [4]:
# genrate data points along this potential
rx = np.linspace(-3, 3, 61)
ry = np.linspace(-3, 3, 61)
RX, RY = np.meshgrid(rx, ry)
Z = U((RX, RY))

General minimisation approach:
$$ q_{k+1} = q_k + \tau_k d_k $$

#### Steepest decent

Direction for minimisation along the gradient:
$$ d_k = -g_k $$
$$ q_{k+1} = q_k - \tau g_k $$

In [10]:
def steep(q, tau):
    """takes a current position q,
    modifies this position by a translation along the gradient
    using a step scale factor tau
    and returns the new postion.
    """
    g = gradient(q)
    return (q[0] - tau*g[0], q[1] - tau*g[1])

#### Conjugate gradient

Direction for minimisation is determined by considering the direction and gradient used in the last step (first step needs to use a different algorithm).

$$ d_k = −g_k + \beta_k d_{k−1} $$

Expression for $\beta$ (Fletcher-Reeves):
$$ \beta_k = \frac{g_k^\mathrm{T} g_k}{g_{k-1}^\mathrm{T} g_{k-1}^\mathrm{\phantom{T}}} $$

In [9]:
def cg(q, d, g, tau):
    gn = gradient(q)
    beta = np.dot(gn, gn) / np.dot(g, g)
    dn = (-gn[0] + beta*d[0], -gn[1] + beta*d[1] )
    return (q[0] + tau*dn[0], q[1] + tau*dn[1]), dn, gn

In [11]:
def minimize(q, emtol = 10e-10,
             nsteps = 1000, tau = 0.09,
             algorithm='steep'):
    energy = [U(q)]; Q = [q]; eps = ['start'] # initialise output
    current_eps = np.inf
    count = 0
    if algorithm == 'steep':
        while current_eps > emtol and count < nsteps:
            q = steep(q, tau)
            Q.append(q)
            energy.append(U(q))
            current_eps = energy[-2] - energy[-1]
            eps.append(current_eps)
            count += 1
        
    elif algorithm == 'cg':
        g = gradient(q); d = (-g[0], -g[1])
        q = steep(q, tau) # One steepest decent step
        Q.append(q)
        energy.append(U(q))
        current_eps = energy[-2] - energy[-1]
        eps.append(current_eps)
        count += 1   
        while current_eps > emtol and count < nsteps:
            q, d, g = cg(q, d, g, tau)
            Q.append(q)
            energy.append(U(q))
            current_eps = energy[-2] - energy[-1]
            eps.append(current_eps)
            count += 1
        
    else:
        print('Algorithm not understood')
        sys.exit(1)
    
    output = {}
    output['Q'] = Q
    output['energy'] = energy
    output['eps'] = eps
    return pd.DataFrame(output)

In [29]:
def cont(name, runs, levels=[0, 1, 4, 10, 40, 100]):
    fig = plt.figure(name)
    C = plt.contour(RX, RY, Z, levels=[0, 1, 4, 10, 40, 100])
    plt.clabel(C, inline=1, fontsize=10)
    for run in runs:
        plt.plot(
            run['Q'].apply(lambda x: x[0]), 
            run['Q'].apply(lambda x: x[1]))
        plt.plot(
            run['Q'].apply(lambda x: x[0]), 
            run['Q'].apply(lambda x: x[1]), 'k.')
    plt.show()